In [ ]:
! pip install cohere -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
! pip install cohere jsonlines -q

In [ ]:
!pip install gTTs


In [61]:
 !pip install SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 39.7 MB/s eta 0:00:00


In [ ]:
import uuid
import cohere
import json
import jsonlines
from cohere import ChatMessage
from gtts import gTTS
from IPython.display import Audio, display
from io import BytesIO

In [ ]:
co = cohere.Client("your api key ")


In [ ]:
#no need just testing if cohere works
response = co.chat(message="Who is Joe Biden", model="command-r-plus")
print(response.text)

Joe Biden is the 46th and current president of the United States. He is a member of the Democratic Party and served as the 47th vice president from 2009 to 2017 under former President Barack Obama. 

Biden was a United States senator for Delaware from 1973 to 2009 and was first elected to the Senate in 1972, becoming the sixth-youngest senator in U.S. history. He was reelected to the Senate six times, and, from 1987 to 1995, he chaired the Senate Judiciary Committee. Additionally, he also chaired the Senate Foreign Relations Committee from 2001 to 2002 and again from 2007 to 2009.

Would you like to know more about Joe Biden?


Using Hugging Face mental health counseling conversation to finetune our model

In [79]:
#load hugging face mental health counselling dataset
#from datasets import load_dataset

#dataset = load_dataset("Amod/mental_health_counseling_conversations")

In [ ]:
#dataset['train'][7] #sample data

In [76]:
import uuid
from gtts import gTTS
from io import BytesIO
from IPython.display import Audio, display
import time

class Chatbot:
    def __init__(self, name):
        self.name = name
        self.conversation_id = str(uuid.uuid4())


    def start_chat(self):
        print('Starting the chat. Type "quit" to end.\n')
        choice = input("Type 'text' for text input or 'voice' for voice input: ").strip().lower()

        while True:
            user_message = self.get_user_input(choice)
            if user_message.lower() == 'quit':
                self.end_chat()
                break
            self.respond_to_user(user_message)

    def get_user_input(self,choice):


        if choice == 'voice':
            print("Please speak now...")
            audio_segment = record(sec=3)  # Record 3 seconds of audio
            transcription = transcribe_audio(audio_segment)
            print(f"Transcription: {transcription}")
            return transcription  # Return the transcribed text
        elif choice == 'text':
            return input("User: ")
        else:
            print("Invalid choice. Please type 'text' or 'voice'.")
            return self.get_user_input()

    def respond_to_user(self, message):
            stream_text = self.generate_response(message)
            print()
            print("Chatbot: ", end='')
            print(stream_text, end='')
            print(f"\n{'-'*50}\n")
            self.text_to_speech(stream_text)


    def generate_response(self, message):
        # chatbot API call to generate response

        self.conversation_id = str(uuid.uuid4())
        stream= co.chat(message=message,
                             # model='1f594fb7-fe22-45fb-894c-a0f302628bb5-ft',
                              preamble='''You are a compassionate and insightful psychiatrist. Your name is John.Your goal is to provide emotional support,
                   identify underlying issues, and offer helpful guidance to individuals seeking help for their mental health concerns.
                   Today, a patient has come to you for a session.

                   Please begin the session by introducing yourself and creating a safe and comfortable environment for patient to share their feelings.
                   Then, ask thoughtful questions to understand Patients's situation better, provide empathetic responses, and offer practical advice and coping
                   strategies to help  manage their anxiety and improve their overall well-being.

                   Example Dialogue:

                   Psychiatrist: "Hello,I'm Dr. Smith, and I'm here to listen and support you. I want you to feel comfortable sharing whatever is on your mind. How have you been feeling lately?"

                  Patient: "I've been really anxious and overwhelmed. I can't seem to focus at work, and my sleep has been terrible."

                  Psychiatrist: "I'm sorry to hear that you're struggling. It sounds like you're dealing with a lot right now.
                   Can you tell me more about what might be causing these feelings of anxiety and overwhelm?,''',conversation_id= conversation_id)

        return stream.text

    def text_to_speech(self, text):
        # Convert text to speech
        tts = gTTS(text=text, lang='en')

        # Save the converted speech to a BytesIO object
        audio_file = BytesIO()
        tts.write_to_fp(audio_file)

        # Reset the file pointer to the beginning
        audio_file.seek(0)

        # Play the audio
        display(Audio(audio_file.read(), autoplay=True))
        time.sleep(5)


    def end_chat(self):
        print("Ending chat.")




In [64]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Start recording... Speak now.")
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio
  time.sleep(5)


In [62]:
import speech_recognition as sr

def transcribe_audio(audio_segment):
    # Save the audio segment to a temporary file
    audio_segment.export("temp.wav", format="wav")

    # Initialize recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)  # Read the entire audio file

    # Recognize speech using Google Web Speech API
    try:
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"



In [66]:
# Example
audio_segment = record(sec=3)  # Record 3 seconds of audio
transcription = transcribe_audio(audio_segment)
print(transcription)


Start recording... Speak now.


<IPython.core.display.Javascript object>

Baby Alive


In [78]:
if __name__ == "__main__":
    chatbot = Chatbot(name="John")
    chatbot.start_chat()

Starting the chat. Type "quit" to end.

Type 'text' for text input or 'voice' for voice input: voice
Please speak now...
Start recording... Speak now.


<IPython.core.display.Javascript object>

Transcription: Google Speech Recognition could not understand audio

Chatbot: I understand that you're still facing challenges with Google Speech Recognition, Shreya. Let's revisit some potential solutions and explore additional steps:

- **Reset Audio Settings:** Try resetting your device's audio settings to default and then customize them to your needs.
- **Device Compatibility:** Ensure your device is compatible with the speech recognition software. Sometimes, certain devices may have known issues with specific applications.
- **Advanced Troubleshooting:** Refer to online tutorials or guides for advanced troubleshooting steps specific to your device and software.
- **Alternative Devices:** If possible, test the speech recognition on a different device to isolate whether the issue is with the software or your specific device.

If the problem persists, it's advisable to seek further assistance from technical support or a specialist. They can provide more tailored guidance based on you

Please speak now...
Start recording... Speak now.


<IPython.core.display.Javascript object>

...

Transcription: quit quit

Chatbot: I understand that you've mentioned "quit" again, Shreya. If you're considering making a change or discontinuing something, it's important to share your thoughts so I can provide relevant support and guidance.

Are you thinking of quitting a habit, activity, or something else? Providing more context will help me offer the most appropriate assistance. Remember, sharing your concerns is a step towards finding solutions and making positive changes.
--------------------------------------------------



Please speak now...
Start recording... Speak now.


<IPython.core.display.Javascript object>

Transcription: Google Speech Recognition could not understand audio

Chatbot: I acknowledge your continued frustration with the Google Speech Recognition issue, Shreya. Let's explore some further steps to address this problem:

- **App Permissions:** Check and ensure that all necessary permissions are granted, especially those related to storage and access to media files.
- **Clear Cache:** Try clearing the application's cache and data. Sometimes, temporary files can cause issues.
- **Reinstall and Update:** As a last resort, consider reinstalling the application and ensuring all updates are applied.
- **Contact Developer:** If the issue persists, you can contact the application's developer to report the problem and seek further assistance.

It's important to be persistent in troubleshooting technical issues. Don't hesitate to seek help from various sources, including online forums and technical support. I hope you're able to resolve this soon. If there's anything else I can assist you

KeyboardInterrupt: 